# Extracting images, calculating AWEI index, and pre-processing with the ground truth

Parameters for extraction are in the following cell

In [1]:
import ee
import geemap
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
import pyproj
import pandas as pd
import time
import pickle

try:
    ee.Initialize(project='ee-mpozdech')
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise
    
def apply_scale_factors(image): # Applies scaling factors for Landstat
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B6').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True)

def image_date(image):
    time_start = ee.Date(image.get('system:time_start'))
    return time_start.format('YYYY-MM-dd').getInfo()


Google Earth Engine has initialized successfully!


Setting the parameters for the script.

In [7]:
# Identifier for lake being looked at
lake_name = 'george'

# Region of Interest to be investigated
coordinates = [149.35704470513303, -34.98, 149.49550371751621, -35.20614998677668]

# Parameters
start_date = '1984-03-16'
end_date = '2022-01-01'

max_cloud_cover = 20         #set % of SAT img that can be cloudy

pxl_fill_threshold = 80      #set % of the SAT img that must have data

merge_tolerance = '15 day'   #set how much of a time-difference there can be between SAT and GT images

pct_correct_threshold = 40   #set % of the AWEI pixels that must also be classified as water by GT to keep said AWEI image. Lower means noisier images

region = ee.Geometry.Rectangle(coordinates)
tol = pd.Timedelta(merge_tolerance)
current_dir = os.getcwd()
lake_dir = os.path.join(current_dir, "lakes", lake_name)
os.makedirs(lake_dir, exist_ok=True)

Loading the datasets.

In [3]:
# Landstat 5 dataset
sat5_dataset = ( 
    ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
    .filterDate(start_date, end_date)
    .filterBounds(region)
    .filterMetadata('CLOUD_COVER','less_than', max_cloud_cover)
)
sat5_dataset = sat5_dataset.map(apply_scale_factors)

sat5_list_num = len(sat5_dataset.aggregate_array('system:index').getInfo())
print('Number of images in the SAT collection: ', sat5_list_num)
sat5_list = sat5_dataset.toList(sat5_dataset.size())

# Landstat 7 dataset
sat7_dataset = ( 
    ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
    .filterDate(start_date, end_date)
    .filterBounds(region)
    .filterMetadata('CLOUD_COVER','less_than', max_cloud_cover)
)
sat7_dataset = sat7_dataset.map(apply_scale_factors)

sat7_list_num = len(sat7_dataset.aggregate_array('system:index').getInfo())
print('Number of images in the SAT collection: ', sat7_list_num)
sat7_list = sat7_dataset.toList(sat7_dataset.size())

# Ground Truth dataset
gt_dataset = ee.ImageCollection('JRC/GSW1_4/MonthlyHistory') \
    .filterBounds(region) \
    .filterDate(start_date, end_date) #only 1 band: 'water'

gt_list_num = len(gt_dataset.aggregate_array('system:index').getInfo())
print('Number of images in the GT collection: ', gt_list_num)
gt_list = gt_dataset.toList(gt_dataset.size())

Number of images in the SAT collection:  217
Number of images in the SAT collection:  418
Number of images in the GT collection:  453


In [4]:
# Visualizations
vis_rgb = {
    'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
    'min': 0.0,
    'max': 0.3,
}

vis_awei = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'blue']
    #'opacity' : 0.3
}

vis_gt = {
    'bands': ['water'],
    'min': 1,
    'max': 2,
    'palette': ['white', 'red'],
    'opacity' : 0.3
}

Loading the GT dataset collection to a dataframe. Date is stored twice to verify the merge later on.

In [5]:
df_gt = pd.DataFrame(columns=['date','gt_img','date_gt'])

for i in range(0,gt_list_num):
    img_id = i
    image = ee.Image(gt_list.get(img_id))
    date = image_date(image)
    
    df_gt.loc[img_id,'date'] = pd.to_datetime(date,format='%Y-%m-%d',errors='coerce',utc=True)
    df_gt.loc[img_id,'gt_img'] = image
    df_gt.loc[img_id,'date_gt'] = date
    
df_gt

,date,gt_img,date_gt
0,1984-04-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-04-01
1,1984-05-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-05-01
2,1984-06-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-06-01
3,1984-07-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-07-01
4,1984-08-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",1984-08-01
...,...,...,...
448,2021-08-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-08-01
449,2021-09-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-09-01
450,2021-10-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-10-01
451,2021-11-01 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",2021-11-01


Loading the SAT5 dataset collection to a dataframe.

In [8]:
df_sat5 = pd.DataFrame(columns=['date','img','pxl_fill','cloud_cover'])

for i in range(0,sat5_list_num): #Change upper bound to be length of post-none drop df
    img_id = i
    image = ee.Image(sat5_list.get(img_id)) #grabs img from dataset list
    date = image_date(image)

    df_sat5.loc[img_id,'date'] = pd.to_datetime(date,format='%Y-%m-%d',errors='coerce',utc=True)

    #compute AWEI index    
    awei = image.expression(
    '4 * (B2 - B5) - (0.25 * B4 + 2.75 * B7)',
    {
        'B2': image.select('SR_B2'),
        'B5': image.select('SR_B5'),
        'B4': image.select('SR_B4'),
        'B7': image.select('SR_B7')
    })
    
    image = image.addBands(awei.rename('awei')) #add calculated index as a band to the SAT image
    df_sat5.loc[img_id,'img'] = image           #save image object to df

    #compute amount of pixels which have some captured data
    band = image.select('SR_B1')    #arbitrary selection of SAT band to check pixel occupancy
    none_pixels = band.mask().eq(0) #masks empty pixels (band value = 'None') with 1, otherwise 0

    #sums how many pixels in region have mask value = 1
    none_pixel_count = none_pixels.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('SR_B1')
    none_count = none_pixel_count.getInfo()

    not_none_pixels = band.unmask().neq(0) #replaces all masked values (empty pixels) with 0

    #sums how many pixels in region have mask value = 1 (prior call inverted said mask)
    not_none_pixel_count = not_none_pixels.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('SR_B1')
    filled_count = not_none_pixel_count.getInfo()

    df_sat5.loc[img_id,'pxl_fill'] = filled_count / (filled_count + none_count) * 100 #computes % of region which has data in it
    
    df_sat5.loc[img_id,'cloud_cover'] = image.get('CLOUD_COVER').getInfo()

df_sat5

,date,img,pxl_fill,cloud_cover
0,1987-09-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0
1,1987-11-21 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,6
2,1988-01-08 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,3
3,1988-01-24 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0
4,1988-02-09 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,13
...,...,...,...,...
212,2004-08-15 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",0.23715,6
213,2004-08-31 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",0.062011,6
214,2006-06-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",0.0,15
215,2007-01-28 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",0.042571,1


Loading the SAT7 dataset collection to a dataframe.

In [9]:
df_sat7 = pd.DataFrame(columns=['date','img','pxl_fill','cloud_cover'])

for i in range(0,sat7_list_num): #Change upper bound to be length of post-none drop df
    img_id = i
    image = ee.Image(sat7_list.get(img_id)) #grabs img from dataset list
    date = image_date(image)

    df_sat7.loc[img_id,'date'] = pd.to_datetime(date,format='%Y-%m-%d',errors='coerce',utc=True)

    #compute AWEI index    
    awei = image.expression(
    '4 * (B2 - B5) - (0.25 * B4 + 2.75 * B7)',
    {
        'B2': image.select('SR_B2'),
        'B5': image.select('SR_B5'),
        'B4': image.select('SR_B4'),
        'B7': image.select('SR_B7')
    })
    
    image = image.addBands(awei.rename('awei')) #add calculated index as a band to the SAT image
    df_sat7.loc[img_id,'img'] = image           #save image object to df

    #compute amount of pixels which have some captured data
    band = image.select('SR_B1')    #arbitrary selection of SAT band to check pixel occupancy
    none_pixels = band.mask().eq(0) #masks empty pixels (band value = 'None') with 1, otherwise 0

    #sums how many pixels in region have mask value = 1
    none_pixel_count = none_pixels.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('SR_B1')
    none_count = none_pixel_count.getInfo()

    not_none_pixels = band.unmask().neq(0) #replaces all masked values (empty pixels) with 0

    #sums how many pixels in region have mask value = 1 (prior call inverted said mask)
    not_none_pixel_count = not_none_pixels.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('SR_B1')
    filled_count = not_none_pixel_count.getInfo()

    df_sat7.loc[img_id,'pxl_fill'] = filled_count / (filled_count + none_count) * 100 #computes % of region which has data in it
    
    df_sat7.loc[img_id,'cloud_cover'] = image.get('CLOUD_COVER').getInfo()

df_sat7

,date,img,pxl_fill,cloud_cover
0,1999-08-10 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,19
1,1999-11-30 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,1
2,2000-02-02 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0
3,2000-04-22 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,8
4,2000-05-08 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,19
...,...,...,...,...
413,2022-01-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",24.844408,16
414,2022-03-25 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",12.390018,19
415,2022-09-24 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",0.333115,8
416,2022-12-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",80.372748,19


Merging the dataframes and ordering them by date.

In [10]:
df_sat57 = pd.concat([df_sat5, df_sat7])
df_sat57 = df_sat57.sort_values('date')
df_sat57

,date,img,pxl_fill,cloud_cover
0,1987-09-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0
143,1987-09-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",0.004941,2
1,1987-11-21 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,6
2,1988-01-08 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,3
3,1988-01-24 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0
...,...,...,...,...
188,2023-09-20 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",88.6671,2
189,2023-10-12 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",24.957316,5
383,2023-10-12 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",12.809099,7
190,2023-11-13 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",75.650687,12


In [12]:
#checkpoint to save and reload data for debugging
df_sat57.to_pickle('df_sat57_pre_fill_drop.pkl')
#df_sat57 = pd.read_pickle('df_sat57_pre_fill_drop.pkl')
#pd.set_option('display.max_rows', 20)
#print(df_sat57)

Dropping all images that had less than the acceptable amount of pixels filled with data.

In [13]:
df_sat57 = df_sat57.drop(df_sat57[df_sat57['pxl_fill'] < pxl_fill_threshold].index) # drops all rows will less than this data
df_sat57.reset_index(drop=True,inplace=True)
df_sat57

,date,img,pxl_fill,cloud_cover
0,1987-09-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0
1,1987-11-21 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,6
2,1988-01-08 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,3
3,1988-01-24 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0
4,1988-02-09 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,13
...,...,...,...,...
282,2017-04-05 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",86.392508,10
283,2017-05-07 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",87.135136,0
284,2017-06-24 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",87.773878,0
285,2017-07-10 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",87.675365,18


Merging the SAT and GT dataframes based on the date within set tolerance.

In [14]:
df_sat57['date'] = df_sat57['date'].apply(lambda x: x.to_pydatetime()) #converts timestamps to datetime
df_gt['date'] = df_gt['date'].apply(lambda x: x.to_pydatetime())

df_sat_gt = pd.merge_asof(left=df_sat57,right=df_gt,on='date',right_index=False,left_index=False,direction='nearest',tolerance=tol) #merge on dates
df_sat_gt = df_sat_gt.dropna()
df_sat_gt

,date,img,pxl_fill,cloud_cover,gt_img,date_gt
0,1987-09-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,"ee.Image({\n ""functionInvocationValue"": {\n ...",1987-10-01
1,1987-11-21 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,6,"ee.Image({\n ""functionInvocationValue"": {\n ...",1987-12-01
2,1988-01-08 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,3,"ee.Image({\n ""functionInvocationValue"": {\n ...",1988-01-01
3,1988-01-24 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,"ee.Image({\n ""functionInvocationValue"": {\n ...",1988-02-01
4,1988-02-09 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,13,"ee.Image({\n ""functionInvocationValue"": {\n ...",1988-02-01
...,...,...,...,...,...,...
282,2017-04-05 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",86.392508,10,"ee.Image({\n ""functionInvocationValue"": {\n ...",2017-04-01
283,2017-05-07 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",87.135136,0,"ee.Image({\n ""functionInvocationValue"": {\n ...",2017-05-01
284,2017-06-24 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",87.773878,0,"ee.Image({\n ""functionInvocationValue"": {\n ...",2017-07-01
285,2017-07-10 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",87.675365,18,"ee.Image({\n ""functionInvocationValue"": {\n ...",2017-07-01


Reproject the GT band using the SAT projection, then add GT band to the SAT img. Saves statistics about the AWEI and GT images to seperate df for merge.

In [ ]:
df_class = pd.DataFrame(columns=['tot_awei_water_pxls','tot_gt_water_pxls','correctly_classified_pxls','pct_correct'])

for i in range(0,len(df_sat_gt)):
    img = df_sat_gt.loc[i,'img']          #gets SAT img object from df
    img_gt = df_sat_gt.loc[i,'gt_img']     #gets GT img object from df
    sat_proj = img.projection().getInfo() #gets SAT crs
    img_gt = img_gt.reproject(sat_proj['crs'],sat_proj['transform']) #transforms GT to SAT projection
    df_sat_gt.loc[i,'gt_img'] = img_gt    #saves reprojected GT image to df
    
    ### AWEI WATER COUNT
    awei = img.select('awei')   #selects awei index band from SAT img
    img_awei_water = awei.gt(0) #gets all values > 0 (classified as water in AWEI)
    #counts number of pixels that were marked as water in AWEI
    awei_water_count = img_awei_water.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('awei')
    df_class.loc[i,'tot_awei_water_pxls'] = awei_water_count.getInfo() #save the number of pixels marked as water by AWEI
    
    ### GT WATER COUNT
    water_classified = img_gt.eq(2) #sets values = 2 (classified water in GT) to 1
    #counts number of pixels that were marked as water in GT
    gt_water_count = water_classified.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('water')
    df_class.loc[i,'tot_gt_water_pxls'] = gt_water_count.getInfo()
    
    ### CORRECT AWEI COUNT
    compare_awei_gt = water_classified.And(img_awei_water) #gets all pixels that were marked as water by both AWEI and GT
    #counts number of pixels correctly classified as water by AWEI
    correctly_classified_water_pxls = compare_awei_gt.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=30,
        maxPixels=1e9
    ).get('water')
    df_class.loc[i,'correctly_classified_pxls'] = correctly_classified_water_pxls.getInfo()
    
    # computes % of AWEI pixels that were also classified as water by GT
    if awei_water_count.getInfo() != 0:
        df_class.loc[i,'pct_correct'] = correctly_classified_water_pxls.getInfo() / awei_water_count.getInfo() * 100
    else:
        df_class.loc[i,'pct_correct'] = 0
        
df_class

Merging the stats df with the img df on index.

In [21]:
df_fin = pd.merge(df_sat_gt,df_class,left_index=True,right_index=True,how='inner')
df_fin.reset_index(drop=True,inplace=True)
df_fin

,date,img,pxl_fill,cloud_cover,gt_img,date_gt,tot_awei_water_pxls,tot_gt_water_pxls,correctly_classified_pxls,pct_correct
0,1987-09-18 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,"ee.Image({\n ""functionInvocationValue"": {\n ...",1987-10-01,30272,13325,5979,19.750925
1,1987-11-21 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,6,"ee.Image({\n ""functionInvocationValue"": {\n ...",1987-12-01,36343,6862,5898,16.22871
2,1988-01-08 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,3,"ee.Image({\n ""functionInvocationValue"": {\n ...",1988-01-01,68,1108,25,36.764706
3,1988-01-24 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,"ee.Image({\n ""functionInvocationValue"": {\n ...",1988-02-01,6333,2371,749,11.826938
4,1988-02-09 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,13,"ee.Image({\n ""functionInvocationValue"": {\n ...",1988-02-01,35080,2371,2359,6.724629
...,...,...,...,...,...,...,...,...,...,...
281,2017-02-16 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",86.746549,4,"ee.Image({\n ""functionInvocationValue"": {\n ...",2017-03-01,97267,101985,88503,90.98975
282,2017-04-05 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",86.392508,10,"ee.Image({\n ""functionInvocationValue"": {\n ...",2017-04-01,91314,77445,69493,76.103336
283,2017-05-07 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",87.135136,0,"ee.Image({\n ""functionInvocationValue"": {\n ...",2017-05-01,87114,0,0,0.0
284,2017-06-24 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",87.773878,0,"ee.Image({\n ""functionInvocationValue"": {\n ...",2017-07-01,82776,0,0,0.0


In [22]:
#checkpoint to save and reload data for debugging
df_fin.to_pickle('df_fin_pre_noise_drop.pkl')
#df_fin = pd.read_pickle('df_fin_pre_noise_drop.pkl')
#pd.set_option('display.max_rows', 20)
#print(df_fin)

Dropping rows where the stats showed low correctness in initial AWEI and GT comparison.

In [23]:
df_fin = df_fin.drop(df_fin[(df_fin['pct_correct'] < pct_correct_threshold) & (df_fin['tot_awei_water_pxls'] > 0)].index) # drops all rows with more than 40% of high-value pixels not captured and there were high-value pixels present
df_fin.reset_index(drop=True,inplace=True)
df_fin

,date,img,pxl_fill,cloud_cover,gt_img,date_gt,tot_awei_water_pxls,tot_gt_water_pxls,correctly_classified_pxls,pct_correct
0,1988-08-03 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,8,"ee.Image({\n ""functionInvocationValue"": {\n ...",1988-08-01,125437,94863,94823,75.594123
1,1988-09-04 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,14,"ee.Image({\n ""functionInvocationValue"": {\n ...",1988-09-01,59115,89672,53486,90.477882
2,1988-10-06 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,0,"ee.Image({\n ""functionInvocationValue"": {\n ...",1988-10-01,130757,59323,59304,45.35436
3,1990-03-02 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,17,"ee.Image({\n ""functionInvocationValue"": {\n ...",1990-03-01,157008,156513,156348,99.579639
4,1990-08-25 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",100.0,6,"ee.Image({\n ""functionInvocationValue"": {\n ...",1990-09-01,164939,158484,158362,96.012465
...,...,...,...,...,...,...,...,...,...,...
171,2015-12-12 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",87.870217,17,"ee.Image({\n ""functionInvocationValue"": {\n ...",2015-12-01,0,335,0,0
172,2016-12-30 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",86.686503,17,"ee.Image({\n ""functionInvocationValue"": {\n ...",2017-01-01,114669,54904,47129,41.100036
173,2017-01-15 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",86.158718,2,"ee.Image({\n ""functionInvocationValue"": {\n ...",2017-01-01,112914,54904,54769,48.505057
174,2017-02-16 00:00:00+00:00,"ee.Image({\n ""functionInvocationValue"": {\n ...",86.746549,4,"ee.Image({\n ""functionInvocationValue"": {\n ...",2017-03-01,97267,101985,88503,90.98975


Adding the GT band to the SAT img, and exporting all SAT imgs.

In [ ]:
for i in range(66,len(df_fin)):
    image = df_fin.loc[i,'img']
    gt_img = df_fin.loc[i,'gt_img']
    image = image.addBands(gt_img.rename('gt')) #add classified gt as a band to the SAT image
    capture_date = image_date(image)

    file_loc = os.path.join(lake_dir, capture_date+'.tif') 

    geemap.ee_export_image(image, filename=file_loc, scale=30, region=region, file_per_band=False)
